<a href="https://colab.research.google.com/github/MOHOAzure/AI_Paint_Online/blob/main/Colab/MyNaiFu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This note is inspired by:

*   https://boards.4channel.org/g/thread/89095460#p89097704
*   https://colab.research.google.com/drive/1kw3egmSn-KgWsikYvOMjJkVDsPLjEMzl
*   https://colab.research.google.com/drive/1AfAmwLMd_Vx33O9IwY2TmO9wKZ8ABRRa

Thanks for your contributions

In [ ]:
#@title ### Stop. Kill any colab process
#@markdown - Killing the colab process will cause the page to report an error, but don't worry, just keep running, don't skip it
import os
os.kill(os.getpid(), 9)

In [ ]:
#@title ### 0. Check GPU

!nvidia-smi

In [ ]:
#@title ### 1. Download NaiFu & My model - from my drive
##@markdown - You could "restart" if takes too much time

from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/naifu
!cp -R /content/drive/MyDrive/stable-diffusion/tools/Naifu/* /content/naifu

!echo "Decompressing..."
!unzip -d /content/naifu /content/naifu/Naifu.zip
!echo "Done."

!cp /content/drive/MyDrive/stable-diffusion/models/[7G]novelai_full+momoko-e+f222.ckpt /content/naifu/models/my-model/model.ckpt

In [ ]:
#@title ### 1. (Alt.) Download NaiFu & Model animefull-final-pruned - from 3rd-party
#@markdown - You could "restart" if takes too much time

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [ ]:
#@title ### 2. Download Dependencies
#@markdown - This step will take a while to complete.
## @markdown - At this time, you can check that model(s) is ready on your Google Drive

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
!/content/naifu/venv/bin/python -m pip install -qq pytorch_lightning==1.7.7

In [ ]:
#@title ### 3. Luanch NaiFu
#@markdown - Waiting for "Application startup complete" and then go to the deployed site
%cd /content/
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/my-model
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969 